In [1]:
# 🧪 Step 2.5: Negative 샘플 생성
import pandas as pd
from pyfaidx import Fasta
import numpy as np
import random
import os

# 경로 설정
FASTA_PATH = "../data/raw/refMrna.fa"
POSITIVE_CSV = "../data/processed/positive_windows.csv"
OUTPUT_CSV = "../data/processed/negative_windows.csv"

# RefSeq 서열 불러오기
fasta = Fasta(FASTA_PATH)

# Positive 샘플 불러오기
positive = pd.read_csv(POSITIVE_CSV)

# Positive 좌표 저장 (중복 방지용)
positive_coords = set()
for _, row in positive.iterrows():
    positive_coords.add((row["Transcript"], row["Position"]))

# Negative 샘플 생성
negatives = []
for _, row in positive.iterrows():
    tid = row["Transcript"]
    if tid not in fasta:
        continue

    seq = str(fasta[tid][:]).upper().replace("T", "U")
    length = len(seq)
    tries = 0
    while tries < 100:
        center = random.randint(25, length - 26)
        if (tid, center) not in positive_coords:
            window = seq[center - 25 : center + 26]
            if len(window) == 51 and set(window).issubset({"A", "C", "G", "U"}):
                negatives.append({
                    "Transcript": tid,
                    "Position": center,
                    "sequence": window,
                    "label": 0
                })
                break
        tries += 1

# 저장
neg_df = pd.DataFrame(negatives)
os.makedirs("../data/processed", exist_ok=True)
neg_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Saved {len(neg_df)} negative samples to {OUTPUT_CSV}")

✅ Saved 20094 negative samples to ../data/processed/negative_windows.csv


## 🧪 Step 2.5: Negative 샘플 생성 (51nt RNA window)

이 단계에서는 Positive 샘플과 동일한 transcript 내에서,  
CLIP binding되지 않은 위치를 랜덤하게 선택하여 Negative 샘플을 생성합니다.

### 📌 주요 처리
- binding되지 않은 위치를 100번까지 random sampling
- window 내 염기 구성이 A, U, G, C만 포함되도록 제한
- Positive와 동일한 형식으로 `sequence`, `Transcript`, `Position`, `label=0` 저장

### 💾 출력 파일
- `../data/processed/negative_windows.csv`

이 데이터는 이후 RNAfold 구조 예측에 사용됩니다.